# Privacy Preserving Machine Learning

Course taught by Aurélien Bellet

Course page: http://researchers.lille.inria.fr/abellet/teaching/private_machine_learning_course.html

# Practical session 3: Differentially Private ERM via Output Perturbation

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.linear_model import LogisticRegression

## Datasets

**You are free to work with any binary classification dataset(s) you like** (you may use more than a single dataset).

It is of course possible to work with the US Census dataset used in previous practicals, but some of the observations may more notable on higher dimensional datasets. You can find datasets for instance on [OpenML](https://www.openml.org/), [UCI](https://archive.ics.uci.edu/ml/index.php), [sklearn](https://scikit-learn.org/stable/modules/classes.html?highlight=datasets#module-sklearn.datasets).

The code below loads a version of the Arcene dataset, which aims to distinguish cancer versus healthy patients from mass-spectrometric data.

In [2]:
X, y = fetch_openml(name='Arcene', version=1, return_X_y=True, as_frame=False)
features_complete = (np.isnan(X).sum(axis=0)) == 0 # features without missing values
X = X[:, features_complete]
n, d = X.shape
print(n, d)

200 10000


## Question 1 (non-private training)

We will work with $\ell_2$-regularized logistic regression. For now, let us consider the non-private setting.

1. What is the Lipschitz constant $L$ of the logistic loss for a data point $(x,y)$? *(Note: we have seen this in the lecture)*. Normalize the dataset to ensure that the logistic loss is 1-Lipschitz for any $(x,y)$.
2. Randomly split the dataset into a training set and a test set.
3. Plot the accuracy of an $\ell_2$-regularized logistic regression classifier on the (fixed) test set with respect to the size of the training set. To do this, you may train the classifier using only the first 10%, 20%, ... of the training set obtained from step 2.

## Question 2 (sensitivity)

Recall the $\ell_2$ regularized logistic regression solves the following problem:
$$\arg\min_{\theta\in\mathbb{R}^p} \frac{1}{n}\sum_{i=1}^n \log(1+e^{-y
(\theta^\top x)}) + \lambda\|\theta\|_2^2$$

1. What is the theoretical upper-bound on the $\ell_2$-sensitivity of the above formulation, based on the result seen in the lecture?
2. We would like to see how large the sensitivity is in practice when we train a logistic regression classifier on two neighboring datasets. Let `n_eval` be the training set size and `n_runs` the number of times we would like to repeat the experiment. For each of the `n_runs` experiments, you should randomly generate a pair of neighboring datasets $D,D'$ of size `n_eval` which differ on a single data point, train logistic regression on $D$ and $D'$, and compute the sensitivity you observe empirically. Implement this procedure.
3. For `n_eval=100`, `n_runs=100` and a value $\lambda$ of your choice, compare the "empirical sensitivity" you observe to the theoretical bound. Explain any discrepancy between the two. **Warning**: scikit-learn solves the logistic regression problem in a slighly different form, see the [documentation](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression). You should make sure that you appropriately convert the value $\lambda$ in the optimization problem above into a value for the parameter `C` of the implementation so that both problems are equivalent.
4. **(bonus)** Investigate what the differing data points (both features and labels) look like in the "best" and "worst" case values of the empirical sensitivity. Investigate how the discrepancy between the empirical and theoretical sensitivity evolves with `n_eval` and $\lambda$. Briefly describe your observations.

## Question 3 (private training via output perturbation)

Assume that you hold the private dataset and you would like to publicly release a logistic regression classifier trained on private dataset in a differentially private manner. We will do this with the DP-ERM via output perturbation approach seen in the lecture.

1. Do you think it is safe to use an empirical estimate of the sensitivity (e.g., as computed in Question 2) instead of a theoretical upper-bound to calibrate the noise? Justify your answer.
2. Implement the DP-ERM via output perturbation approach. The function should take as input a (private) training dataset, a value of regularization parameter $\lambda$, and privacy parameters $\epsilon$ and $\delta$, and return a model trained on the private dataset in an $(\epsilon,\delta)$-DP manner.
3. Experiment with your DP-ERM implementation for different training set size, regularization parameter and privacy parameter $\epsilon$ (you can set $\delta$ to a fixed value by following the recommendation seen in the second lecture). Compare the test accuracy of the resulting model to the non-private case. Discuss the trade-offs you obtain.
4. **(bonus)** As discussed in the lecture, the cost of differential privacy for machine learning can be measured as the number of additional data points needed to match the test accuracy of the non-private case. Illustrate this with an experiment.

## Question 4 (membership inference attack)

Machine learning models can leak information about the individual data points on which they were trained. Given a data point and some access to target model (e.g., to its parameters or only to predictions made by the model), a *membership inference attack* aims to predict whether the data point was in the model's training dataset.

Here, we consider the simplest setting for membership inference attacks, where the attacker has access to the parameters of the trained model ("white-box") as well as a separate set of data points from the same distribution as the training set (you may use the test set of previous questions for this purpose).

1. Playing the role of the attacker with access to a target model and a separate set of points (but not the training set!), implement a simple membership inference attack which uses the soft predictions (class probabilities) of the target model on a given point to predict whether it was in the training dataset. Soft predictions are accessible by calling the method `predict_proba`.
2. Evaluate the performance of your attack on non-private logistic regression models trained on a subset of the training set. Investigate how the performance varies with the size of the training set and the regularization parameter.
3. Study the effectiveness of the DP-ERM approach implemented in Question 2 to mitigate the effectiveness of the membership inference attack.

## Bonus Question 1 (objective perturbation)

Beyond output perturbation, the work of [Chaudhuri et al. (2011)](https://www.jmlr.org/papers/volume12/chaudhuri11a/chaudhuri11a.pdf) introduces a DP-ERM algorithm based on *objective perturbation*. Implement this algorithm for $\ell_2$-regularized regression and compare the results with those of output perturbation.

To optimize the perturbed objective, you may use a solver from `scipy.optimize.minimize` (such as L-BFGS, which is used by default by scikit-learn to fit logistic regression). You will need to implement a function that computes the objective value and the gradient at a given point: for this, you can adapt [the one used in scikit-learn for the non-perturbed objective](https://github.com/scikit-learn/scikit-learn/blob/0fb307bf39bbdacd6ed713c00724f8f871d60370/sklearn/linear_model/_logistic.py#L84).

## Bonus Question 2 (inference attacks on richer models)

Membership inference attacks are especially effective on expressive models.

1. Evaluate the performance of your membership inference attack on models richer than $\ell_2$-regularized logistic regression, such as kernel SVM, random forests and neural networks (multi-layer perceptrons).
2. Compute the empirical sensitivity of these models as done in Question 2. Compare the results across the different families of models. Is (empirical) sensitivity a good indicator of whether a model is prone to membership inference attacks?

## Bonus Question 3 (more realistic membership inference attacks)

The membership inference attack implemented so far relies on two strong assumptions: "white-box" access to the target model and above all access to a separate dataset that follows the same distribution as the training data. This is unlikely to be the case in practice. Membership inference attacks of the literature aim to lift these assumptions by only relying on "black-box" access to the target model (where the attacker can only query the model without access to its parameters) and no additional data, see for instance the work of [Shokri et al. (2017)](https://www.cs.cornell.edu/~shmat/shmat_oak17.pdf).

Using ideas from the literature (such as those in the paper above), implement a more realistic membership inference attack and evaluate its performance.